# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

In [10]:
acfpt = Website("https://acfpt.com.au/")
acfpt.links

['#content',
 '#',
 'https://acfpt.com.au/',
 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/',
 'https://acfpt.com.au/national-qualifications/',
 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/frequently-asked-questions/',
 'https://acfpt.com.au/courses/',
 'https://acfpt.com.au/courses/sis30321-certificate-iii-in-fitness/',
 'https://acfpt.com.au/courses/hlt33021-certificate-iii-in-allied-health-assistance/',
 'https://acfpt.com.au/courses/sis40221-certificate-iv-in-fitness-personal-trainer/',
 'https://acfpt.com.au/courses/sis40221-certificate-iv-in-fitness-fast-track/',
 'https://acfpt.com.au/courses/dual-qualification-in-fitness-sis30321-sis40221-new/',
 'https://support.acfpt.com.au',
 'https://acfpt.com.au/student-support/',
 'https://acfpt.com.au/student-support/policies-and-terms-conditions/austudy-abstudy/',
 'https://acfpt.com.au/work-placement/',
 'https://acfpt.com.au/student-support/policies-and-terms-condi

In [11]:
get_links("https://acfpt.com.au/")

{'links': [{'type': 'about page',
   'url': 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/'},
  {'type': 'courses page', 'url': 'https://acfpt.com.au/courses/'},
  {'type': 'contact page', 'url': 'https://acfpt.com.au/contact/'},
  {'type': 'work placement page',
   'url': 'https://acfpt.com.au/work-placement/'},
  {'type': 'blog page', 'url': 'https://acfpt.com.au/blog/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://acfpt.com.au/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/'}, {'type': 'courses page', 'url': 'https://acfpt.com.au/courses/'}, {'type': 'contact page', 'url': 'https://acfpt.com.au/contact/'}, {'type': 'work placement', 'url': 'https://acfpt.com.au/work-placement/'}, {'type': 'blog', 'url': 'https://acfpt.com.au/blog/'}]}
Landing page:
Webpage Title:
Australian College of Fitness and Personal Training - ACFPT
Webpage Contents:
Skip to content
Search for:
Home
About Us
National Qualifications
Frequently Asked Questions
Courses
Certificate III
SIS30321 Certificate III in Fitness (Group Exercise Instructor & Gym Instructor)
HLT33021 Certificate III in Allied Health Assistance
Certificate IV
Become a Personal Trainer with SIS40221 Certificate IV in Fitness
Fast Track Personal Training Course
Dual Qualification
Dual Qualification in Fitness SIS30321 + SIS40221
Support
Student Support
Austudy & Abstudy
Work Placemen

In [28]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure in Farsi language about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Australian College of Fitness and Personal Training", "https://acfpt.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/'}, {'type': 'courses page', 'url': 'https://acfpt.com.au/courses/'}, {'type': 'contact page', 'url': 'https://acfpt.com.au/contact/'}, {'type': 'work placement page', 'url': 'https://acfpt.com.au/work-placement/'}, {'type': 'blog page', 'url': 'https://acfpt.com.au/blog/'}]}


'You are looking at a company called: Australian College of Fitness and Personal Training\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nAustralian College of Fitness and Personal Training - ACFPT\nWebpage Contents:\nSkip to content\nSearch for:\nHome\nAbout Us\nNational Qualifications\nFrequently Asked Questions\nCourses\nCertificate III\nSIS30321 Certificate III in Fitness (Group Exercise Instructor & Gym Instructor)\nHLT33021 Certificate III in Allied Health Assistance\nCertificate IV\nBecome a Personal Trainer with SIS40221 Certificate IV in Fitness\nFast Track Personal Training Course\nDual Qualification\nDual Qualification in Fitness SIS30321 + SIS40221\nSupport\nStudent Support\nAustudy & Abstudy\nWork Placement\nPolicies and Terms & Conditions\nApplication for enrolment\nWithdrawal of Enrolment and Refund Policy\nLanguage Literacy and Numeracy Policy 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("ACFPT", "https://acfpt.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/'}, {'type': 'courses page', 'url': 'https://acfpt.com.au/courses/'}, {'type': 'work placement page', 'url': 'https://acfpt.com.au/work-placement/'}, {'type': 'blog page', 'url': 'https://acfpt.com.au/blog/'}, {'type': 'contact page', 'url': 'https://acfpt.com.au/contact/'}]}


# ACFPT Brochure

---

## Welcome to ACFPT

**Australian College of Fitness and Personal Training (ACFPT)** is a premier institution dedicated to training individuals in the fields of fitness and allied health. As a registered training organisation recognized under the Australian Qualifications Framework, ACFPT offers nationally accredited courses, ensuring learners receive a quality education that meets stringent national standards.

---

### Our Courses

We empower our students with a selection of comprehensive courses designed to meet the needs of today's fitness industry:

- **Certificate III in Fitness (SIS30321)**: 
  - Become a Group Exercise & Gym Instructor.
  - Gain the essential skills to help others reach their fitness goals.

- **Certificate IV in Fitness (SIS40221)**:
  - Step into a rewarding career as a Personal Trainer.
  - Obtain the knowledge to work independently in various fitness environments.

- **Fast Track Certificate IV in Fitness**:
  - Accelerate your training journey and become qualified to take on clients sooner.

- **Certificate III in Allied Health Assistance (HLT33021)**:
  - Support allied health professionals with knowledge and practical skills.

Explore our full range of courses [here](#).

---

### Company Culture

At ACFPT, we believe in creating an accessible and supportive learning environment. Our unique online training model allows students to study at their own pace, ensuring flexibility and convenience. Our trainers are not only industry experts but also hold relevant academic qualifications, which enriches the learning experience.

We are dedicated to student success and offer 24/7 support along with one-on-one consultations. Our refund policy ensures a risk-free enrollment process, with a 100% refund available within the first 14 days if the course doesn't meet expectations.

---

### Our Customers

Our students range from fitness enthusiasts looking to turn their passion into a profession to current health professionals wanting to expand their qualifications. We cater to a diverse array of individuals aspiring to work in gyms, health clubs, or even open their own businesses.

---

### Career Opportunities

ACFPT is not just a place to learn but a community where you can build a rewarding career. We continually seek passionate educators and industry professionals to join our team. If you have the expertise and enthusiasm for fitness education, we would love to hear from you!

---

### Get in Touch

For inquiries or to enroll in our courses, you can contact us at:

- **Phone**: 1300 511 455  
- **Email**: enquiries@acfpt.com.au  
- **Address**: PO BOX 15367, City East QLD 4002  

Follow us on [Facebook](#), [Twitter](#), [Instagram](#), and [Pinterest](#) for the latest updates!

---

Help us shape the future of fitness. Join ACFPT today and step confidently into your dream career!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("ACFPT", "https://acfpt.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://acfpt.com.au/about-australian-college-of-fitness-and-personal-training/'}, {'type': 'courses page', 'url': 'https://acfpt.com.au/courses/'}, {'type': 'contact page', 'url': 'https://acfpt.com.au/contact/'}, {'type': 'work placement page', 'url': 'https://acfpt.com.au/work-placement/'}, {'type': 'blog page', 'url': 'https://acfpt.com.au/blog/'}]}



# بروشور خنده‌دار ACFPT: دانشکده استرالیایی تناسب اندام و آموزش شخصی

### 🏋️‍♂️ درباره ما:
ما در ACFPT با اعتقاد به این که "تنها به یک گلابی زرد نمی‌توانیم تناسب اندام پیدا کنیم"، به آموزش تناسب اندام و پرورش مربی‌های آینده می‌پردازیم! آیا شما هم آرزو دارید به سادگی با تمرین کردن به دیگران کمک کنید و در عین حال مشغول فوتبال بازی کردن باشید؟ ما دوره‌های گوناگونی داریم که خوشبختانه سوتی‌هایی که در جلسات آنلاین ما پیش می‌آید، به سادگی شما را به سطح جهانی می‌رساند!

---

### 🎓 دوره‌های ما:
- **گواهی‌نامه III در تناسب اندام (SIS30321)**:
  - یاد بگیرید چگونه گروه‌ها را به طرز خنده‌دار و کارآمد آموزش دهید. ممکن است در این دوره متوجه شوید که این گروه از افراد شایسته‌ترین مربیان دنیا خواهند بود – یا حداقل شادترین‌ها!
  
- **گواهی‌نامه IV در تناسب اندام (SIS40221)**:
  - با این مدرک می‌توانید رئیس خودتان باشید! اینجا هیچ کسی به شما نخواهد گفت که کجا تمرین کنید. به سادگی می‌توانید با تکیه بر دانش و مهارتتان، باشگاه خودتان را راه‌اندازی کنید یا در یک باشگاه بزرگ حرفه‌ای فعالیت کنید.

- **دوره فست‌تراک تناسب اندام**:
  - آیا می‌خواهید سریعتر از فکر کردن درباره رژیم غذایی لاغری به مربی شوید؟ این دوره سریع انتخاب مناسبی است! 

---

### 🎉 فرهنگ سازمانی:
در ACFPT ما نه تنها درس می‌دهیم، بلکه یاد می‌گیریم که چطور با خنده کار کنیم! تیم ما کاملاً آنلاین است و با شما مانند یک قهرمان واقعی رفتار می‌کند. شما مدام می‌توانید با مربیانتان در ارتباط باشید و تست‌های سوتی روز را به اشتراک بگذارید.

---

### 💼 شغل و فرصت‌های شغلی:
آیا به دنبال شغف واقعی هستید؟ ما استخدام مربیانی را که توانایی گلابی و انسان بودن را داشته باشند، بعد از دوره‌های آموزشی‌امان تشویق می‌کنیم. البته، مهارت در حرکات ورزشی به جای حرکات ژل ضدعفونی برای دست‌ها هم از دیگر الزامات ماست!

---

### 🤔 سوال دارید؟
اگر سوالی درباره اینکه آیا گلابی‌ها هم می‌توانند مربی شوند، دارید، با ما در ارتباط باشید! ما اینجا هستیم تا شما را به بهترین و جذاب‌ترین مربی تبدیل کنیم!

### 📞 تماس با ما:
برای اطلاعات بیشتر با شماره 1300 511 455 تماس بگیرید یا امانتجویانه ایمیل بزنید به enquiries@acfpt.com.au. ما مطمئنیم که حوصله‌تان سر نمی‌رود!

--- 

**آیا آماده‌اید که با ما شروع کنید؟ ثبت‌نام کنید، و حقایق خنده‌داری که در راه یادگیری پیدا خواهید کرد را کشف کنید!**

---



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>